<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/IV_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def IV_analyzer(abf,measure_windows={'IV_Early':(16, 30),'IV_Steady_State':(100,120)},to_plot=False):
    'Takes the abf data and calculates the mean and peak for each window.'
    'Optionaly sweeps with APs can be filtered out using spike_threshold.'
    'Only sweeps passing the QC filter are used. Returns a list of dicts with'
    'each dict corresponding to one anaysis window reporting the time range, peak etc.'

    # IV_stats = []
    # theta, offset, correct_ch1 = predict_telegraph(abf)
    
    results = {}


    abf = abf_or_name(abf)



    if to_plot:
        fig, axs = plt.subplots(len(measure_windows),2,figsize = [9,6] )
        # if len(measure_windows.keys())==1: axs = [axs]

    for w_i in range(len(measure_windows.keys())):
        w = list(measure_windows.keys())[w_i]
        window_stats = {}
        start = measure_windows[w][0]/1000 # ms to s
        stop = measure_windows[w][1]/1000
        delta_t = stop-start
        flank_neg = start - (delta_t)*0.5
        flank_pos = stop + (delta_t)*0.5
        full_plot_range_ind =   np.logical_and(  abf.sweepX>flank_neg, abf.sweepX<flank_pos )
        spikeless_sweeps = []
        for s in abf.sweepList:
            if to_plot: s_shift_t = delta_t*0.03*s
            if to_plot: s_shift_I = 100*s
            abf.setSweep(s)
            time_abrg = abf.sweepX[full_plot_range_ind]
            I_abrg = abf.sweepY[full_plot_range_ind]

            time_abrg = abf.sweepX[full_plot_range_ind]
            I_abrg = abf.sweepY[full_plot_range_ind]
            analysis_range = np.logical_and(  abf.sweepX>start, abf.sweepX<stop )
            time_analysis = abf.sweepX[analysis_range]
            I_analysis = movmean( abf.sweepY[analysis_range], 4)
            dIdt = movmean(np.diff(I_abrg, prepend=I_abrg[0]),1)


        I_peak_l = []
        I_mean_l = []
        V_stim_l = []

        # measured v_hold
        abf.setSweep(sweepNumber=0, channel=1)
        v_hold = round(abf.sweepY[0]*10)/10
        abf.setSweep(sweepNumber=0, channel=0)
        
        for s in abf.sweepList :
            abf.setSweep(s)
            s_shift_t = delta_t*0.03*s * 0
            s_shift_I = 100*s * 0

            time_abrg = abf.sweepX[full_plot_range_ind]
            I_abrg = abf.sweepY[full_plot_range_ind]
            analysis_range = np.logical_and(  abf.sweepX>start, abf.sweepX<stop )
            time_analysis = abf.sweepX[analysis_range]
            I_analysis = abf.sweepY[analysis_range]
            

            # Peaks
            if to_plot: axs[w_i,0].plot( time_abrg+s_shift_t, I_abrg+s_shift_I,'k')
            
            I_mean = np.median( I_analysis )
            if to_plot: axs[w_i,0].plot( time_analysis+s_shift_t, (I_mean*np.ones_like(time_analysis))+s_shift_I,'r' )            
            
            local_I_mean = movmean(I_analysis,100)
            delta_i = I_analysis-I_mean
            peak_ind = np.where(abs(delta_i) == np.max(abs(delta_i)))[0][0]

            I_peak = delta_i[peak_ind]
            t_peak = time_analysis[peak_ind]
            if to_plot: axs[w_i,0].scatter( t_peak+s_shift_t, I_peak, color='m' )

            command_trace = abf.sweepC + v_hold
            # print('vh, cmd',v_hold, command_trace[analysis_range][-1] )

            I_peak_l.append(I_peak)
            I_mean_l.append(I_mean)

            command_v = round(np.median( command_trace[analysis_range])/10)*10
            V_stim_l.append(command_v)

        # print('I_peak_l', I_peak_l)
        # print('I_mean_l', I_mean_l)
        # print('V_stim_l', V_stim_l)

        window_stats['range'] = (start*1000,stop*1000)
        window_stats['I_peak'] = I_peak_l
        window_stats['I_mean'] = I_mean_l
        window_stats['V_stim'] = V_stim_l


        results[w] = window_stats
        if to_plot:
            axs[w_i,1].scatter(window_stats['V_stim'],window_stats['I_peak'] ,color='magenta' )
            axs[w_i,1].scatter(window_stats['V_stim'],window_stats['I_mean'] ,color='red' )
            axs[w_i,1].spines['left'].set_position('zero')
            axs[w_i,1].spines['bottom'].set_position('zero')

    # plt.show()
    try:    os.makedirs('SavedFigs/IV_Curves/')
    except:     None
    plt.savefig( 'Saved_Figs/IV_Curves/IV_Curves_' + abf.abfID +'.png',dpi=600)
 
    return results


# fn = 'my_ephys_data/2022_08_12_RNF182/2022x08x12_RNF182_E4KI_F_P251_s001_c002_CA3xPOS_0002.abf'
# IV_analyzer
# results = IV_analyzer(fn,to_plot=True)

# # print(results)